In [ ]:
#Carregar Bibliotecas

import keras
from keras.layers import Input,Dense,Flatten,Dropout,merge,Reshape,Conv2D,MaxPooling2D,UpSampling2D,Conv2DTranspose,Activation
from keras.layers.normalization import BatchNormalization
from keras import regularizers
from keras.models import Sequential, Model
from keras.utils.vis_utils import plot_model
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adadelta, RMSprop,SGD,Adam
from keras import backend as K

import numpy as np

import tensorflow as tf

import os

import cv2

import skimage.transform

import matplotlib
import matplotlib.pyplot as plt

matplotlib.interactive(True)

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


from utils import get_initial_weights
from layers import BilinearInterpolation

In [ ]:
###########################################################################################################################
#########################                                  TO DO                                  ######################### 
###########################################################################################################################
#->x TROCAR LIN E COL ONDE TEM 60!!!

In [ ]:
#Carregar Dados

#x_train = np.load('/home/natan/Python_Tutoriais/Test_Tcc/IXI_T1_train.npy')    
#x_train = np.load('/home/natan.andrade/Python_Tutoriais/Test_Tcc/IXI_T1_train.npy')
x_train = np.load('c:/Users/Natan/Documents/Universidade/3_TCC_Eng_Biomed/TCC_Novos/TCC_Experimentos/IXI_T1_train.npy')
np.shape(x_train)
#x_test = np.load('/home/natan/Python_Tutoriais/Test_Tcc/IXI_T1_valid.npy')
#x_test = np.load('/home/natan.andrade/Python_Tutoriais/Test_Tcc/IXI_T1_valid.npy')        
x_test = np.load('c:/Users/Natan/Documents/Universidade/3_TCC_Eng_Biomed/TCC_Novos/TCC_Experimentos/IXI_T1_valid.npy')
np.shape(x_test)

#x_train = np.load('/tmp/natan/Test_TCC/test.npy')    
#x_test = np.load('/tmp/natan/Test_TCC/valid.npy')

#x_test = np.vstack((x_test,x_train[4:7,:]))
#x_train = np.delete(x_train, [i for i in range(4,7)], axis=0)


x_train = np.reshape(x_train, (len(x_train), 119, 119))  # adapt this if using `channels_first` image data format
np.shape(x_train)

x_test = np.reshape(x_test, (len(x_test), 119, 119))  # adapt this if using `channels_first` image data format
np.shape(x_test)

x_train = skimage.transform.resize(x_train,(600,60,60,1))
np.shape(x_train)

x_test = skimage.transform.resize(x_test,(88,60,60,1))
np.shape(x_test)


x_train = np.reshape(x_train, (len(x_train), 60, 60, 1))  
np.shape(x_train)

x_test = np.reshape(x_test, (len(x_test), 60, 60, 1))  
np.shape(x_test)

In [ ]:
batch_size = 128
aux_epoch = 100
epochs = aux_epoch
inChannel = 1
lin, col = 60, 60

#input_img = Input(shape = (x, y, inChannel))

input_img = Input(shape=(lin, col, 1))

#conv1 
x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_img) #28 x 28 x 32
np.shape(x)

#conv1 
x = BatchNormalization()(x)
np.shape(x)

#conv1 
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
np.shape(x)

#conv1
x = BatchNormalization()(x)
np.shape(x)

#pool1 
x1 = MaxPooling2D(pool_size=(2, 2))(x) #14 x 14 x 32
np.shape(x1)

#conv2 
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x1) #14 x 14 x 64
np.shape(x)

#conv2 
x = BatchNormalization()(x)
np.shape(x)

#conv2 
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
np.shape(x)

#conv2 
x = BatchNormalization()(x)
np.shape(x)

#pool2 
x = MaxPooling2D(pool_size=(2, 2))(x) #7 x 7 x 64
np.shape(x)

#conv3 
x = Conv2D(128, (3, 3), activation='relu', padding='same')(x) #7 x 7 x 128 (small and thick)
np.shape(x)

#conv3 
x = BatchNormalization()(x)
np.shape(x)

#conv3 
x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
np.shape(x)
#conv3 
encoded = BatchNormalization()(x)
np.shape(encoded)


In [ ]:
#Questões:
# Vai aprender a não transformar o batch -> Função identidade!
# Vai aprender a registrar todas as imagens funcionando ainda melhor como algo PCA ...
# Dessa maneira existira algum aprendizado em relação as informações do início que fazem com que 
# consiga-se obter um code mais potente?

sampling_size = (15,15)
locnet = Flatten()(encoded)
locnet = Dense(50)(locnet)
locnet = Activation('relu')(locnet)
weights = get_initial_weights(50)
locnet = Dense(6, weights=weights)(locnet)
    
x = BilinearInterpolation(sampling_size)([x1, locnet])


In [ ]:
#decoder
#conv4 
#x = Conv2D(64, (3, 3), activation='relu', padding='same')(encoded) #7 x 7 x 128
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x) #7 x 7 x 128
np.shape(x)

#conv4 
x = BatchNormalization()(x)
np.shape(x)

#conv4 
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
np.shape(x)

#conv4 
x = BatchNormalization()(x)
np.shape(x)

#up1 
x = UpSampling2D((2,2))(x) # 14 x 14 x 128
np.shape(x)

#conv5 
x2 = Conv2D(32, (3, 3), activation='relu', padding='same')(x) # 14 x 14 x 64
np.shape(x2)

#conv5 
x = BatchNormalization()(x)
np.shape(x)

#conv5 
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
np.shape(x)

#conv5 
x = BatchNormalization()(x)
np.shape(x)

#up2 
x = UpSampling2D((2,2))(x) # 28 x 28 x 64
np.shape(x)

# Acrescentar o Spacial Transform depois do Encoder junto com o decoder!
#Questões:
# Vai aprender a não transformar o batch -> Função identidade!
# Vai aprender a registrar todas as imagens funcionando ainda melhor como algo PCA ...
#sampling_size = (60,60)
#locnet = Flatten()(x2)
#locnet = Dense(50)(locnet)
#locnet = Activation('relu')(locnet)
#weights = get_initial_weights(50)
#locnet = Dense(6, weights=weights)(locnet)
    
#x = BilinearInterpolation(sampling_size)([x, locnet])

#decoded 
decoded = Conv2D(1, (2, 2), activation='sigmoid', padding='same')(x) # 28 x 28 x 1
np.shape(decoded)

In [ ]:
encoder = Model(input_img, encoded)
autoencoder = Model(input_img, decoded)
autoencoder.compile(loss='mean_squared_error', optimizer = RMSprop())

autoencoder.summary()


In [ ]:
from keras.utils import plot_model
plot_model(autoencoder,to_file='TCC_CNN_Decoder_STN.png',show_shapes=True)

In [ ]:
autoencoder_train=autoencoder.fit(x_train, x_train, batch_size=batch_size,epochs=epochs,verbose=1,
                                    validation_data=(x_test, x_test))

In [ ]:


loss = autoencoder_train.history['loss']
val_loss = autoencoder_train.history['val_loss']
epochs = range(aux_epoch)
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()




In [ ]:
pred = autoencoder.predict(x_test)
a = 8 # Anda de 4 em 4 por paciente
n = 4  
plt.figure()
for i in range(0,n):
    ax = plt.subplot(2, n, i + 1)
    #Test is input
    plt.imshow(x_test[a+i].reshape(lin,col))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(pred[a+i].reshape(lin , col))
    #decoded_imgs
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
#
#
#

plt.show()


In [ ]:
#encoder.save('encoder_weights.h5')
#decoder.save('decoder_weights.h5')
#autoencoder.save('test_diferente_ae_weights.h5')

In [ ]:
#output_STN = autoencoder.get_layer('bilinear_interpolation_1').output
#np.shape(output_STN)
#STN_function = K.function([x1], [output_STN])
#STN_function
#np.shape(STN_function)

#x_test = np.reshape(x_test, (len(x_test), 60, 60, 1))  
#np.shape(x_test)

#x_test_sample = np.copy(x_test[0:9,:,:,:])
#np.shape(x_test_sample)

#Fazer a saída da entrada até x1 e depois fazer a saida do output_STN ...

#image_result = STN_function([x_test_sample])
#np.shape(image_result)

In [ ]:
#Imagem Teste
#plt.figure()
#plt.imshow(x_test[2].reshape(60,60))
#plt.gray()

#Imagem Transformada
#a = image_result[0]
#np.shape(a)
#b = a[2]
#np.shape(b)
#plt.figure()
#plt.imshow(b.reshape(60,60))
#plt.gray()

In [ ]:
#plot_mnist_grid(x_train)

#plot_mnist_grid(x_train, STN_function)

In [ ]:
'''
#pip install
from keras.layers import Input, Dense
from keras.models import Model
from keras import regularizers
from keras.datasets import mnist
from keras.models import Sequential
#import GraphViz
import keras
from keras.layers import Conv2D, MaxPooling2D, Input, Dense, Flatten
from keras.models import Model
from keras.utils.vis_utils import plot_model
import numpy as np
import tensorflow as tf
import os
import cv2
from keras.layers import Input,Dense,Flatten,Dropout,merge,Reshape,Conv2D,MaxPooling2D,UpSampling2D,Conv2DTranspose
from keras.layers.normalization import BatchNormalization
from keras.models import Model,Sequential
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adadelta, RMSprop,SGD,Adam
from keras import regularizers
from keras import backend as K
import skimage.transform
import matplotlib.pyplot as plt
import matplotlib
matplotlib.interactive(True)
from keras.models import load_model

#x_test = np.load('/home/natan/Python_Tutoriais/Test_Tcc/IXI_T1_valid.npy')
#x_test = np.load('/home/natan.andrade/Python_Tutoriais/Test_Tcc/IXI_T1_valid.npy')
x_test = np.load('c:/Users/Natan/Documents/Universidade/3_TCC_Eng_Biomed/TCC_Novos/TCC_Experimentos/IXI_T1_valid.npy')
x_test = np.reshape(x_test, (len(x_test), 119, 119))  # adapt this if using `channels_first` image data format

x_test = skimage.transform.resize(x_test,(88,60,60))
x_test = np.reshape(x_test, (len(x_test), 60, 60,1))

#x_test = skimage.transform.resize(x_test,(88,120,120))
#x_test = np.reshape(x_test, (len(x_test), 120, 120,1))
np.shape(x_test)

lin, col = 60,60

#autoencoder = load_model('ae_weights.h5')  

autoencoder = load_model('test_diferente_ae_weights.h5')


pred = autoencoder.predict(x_test)

n = 4  
plt.figure()
for i in range(0,n):
    ax = plt.subplot(2, n, i + 1)
    #Test is input
    plt.imshow(x_test[i].reshape(lin,col))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(pred[i].reshape(lin , col))
    #decoded_imgs
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
#
#
#

plt.show()
'''